# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [2]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [27]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample

data = {'table': ['employee', 'salary', 'studies'],
        'definition': ['employee_name, employee_id', 'salary_amount', 'degree']
}
df = pd.DataFrame(data)
print(df)

      table                  definition
0  employee  employee_name, employee_id
1    salary               salary_amount
2   studies                      degree


In [28]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [29]:
print(text_tables)

employee: employee_name, employee_id
salary: salary_amount
studies: degree


In [40]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
Create the SQL querry.
###User Questyion:
{question}
"""


In [41]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question='Give me the name and id of the employee with the highest salary')

In [42]:
print(return_OAI(pqt1))

### Necessary Tables
```json
{
  "tables": ["employee", "salary"]
}
```

### SQL Query
```sql
SELECT e.employee_name, e.employee_id
FROM employee e
JOIN salary s ON e.employee_id = s.employee_id
WHERE s.salary_amount = (SELECT MAX(salary_amount) FROM salary);
```


In [38]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question='Give me the name and id of the employee with the highest salary')

In [39]:
print(return_OAI(pqt3))

```json
{
    "tables": ["employee", "salary"]
}
```


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

Report

The gpt-3.5-turbo model is smart enough to be able to understand what we want and where to find it. It can also give an output in form of JSON and SQL if explicitly asked to do so, which is quite useful.

